In [25]:
def computeBM25Score(R,S,idf,tf,k,b,avgDL):
    S = S.split()
    R = R.split()
    score = 0
    for ws in S:
        numerator = tf[ws] * (k + 1)
        denominator = tf[ws] + k * (1- b + b * (len(R)/avgDL))
        score += computeIdf(ws,idf)  * numerator/denominator
    return score 

In [26]:
def computeIdf(word,idf):
    if word in idf:
        #print(idf[word])
        return idf[word]
    else:
        return 1

In [27]:
def readFile(path):
    file = open(path,"r")
    sentences = []
    for line in file:
        line = line.strip()
        if line.strip():
            if line == "@highlight":
                break
            sentences.append(line)
            
    file.close()
    return sentences

In [28]:
def stopping(sentences):
    stoppedSent = []
    stopWords =[]
    file = open("stop-word-list.txt", 'r')
    for line in file:
        stopWords.append(line.strip())
    for s in sentences:
        temp = s.split()
        sent = ""
        for w in temp:
            if w not in stopWords:
                sent += w + " "
        stoppedSent.append(sent)
    return stoppedSent
    

In [29]:
from nltk.stem import PorterStemmer 

def stemming(sentences):
    stemmedSent = []
    ps = PorterStemmer()
    for s in sentences:
        temp = s.split()
        stemmedWord = ""
        for w in temp:
            stemmedWord += ps.stem(w) + " "
        stemmedSent.append(stemmedWord)
    return stemmedSent
            
    

In [30]:
def createSimilarityMatrix(sentences,idf,k,b):
    BM25_scores = [[0 for x in range(len(sentences))] for y in range(len(sentences))]
    avgDL = computeAvgDL(sentences)
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i == j:
                BM25_scores[i][j] = 1
            else:
                
                tf = generateTF(sentences[i],sentences[j])
                BM25_scores[i][j] = computeBM25Score(sentences[i],sentences[j],idf,tf,k,b,avgDL)
    #print(BM25_scores)
    return BM25_scores

In [31]:
def computeAvgDL(sentences):
    sum =  0
    for s in sentences:
        sum += len(s.split())
    avg = sum/len(sentences)
    return avg

In [32]:
def generateTF(R,S):
    tf = {}
    #print(R,S)
    S = S.split()
    R = R.split()
    for ws in S:
        if ws in R:
            if ws in tf:
                tf[ws] += 1
            else:
                tf[ws] = 1
        else:
            tf[ws] = 0
    #print(tf)
    return tf

In [33]:
import random
def textRank(sentences,BM25_Score,d):
    ws = {}
    #print(BM25_Score)
    #assigning initial pagerank 
    for i in range(len(sentences)):
        ws[i] = 1/len(sentences)
    currSum = 0
    prevSum = -1
    
    count = 0
    while((currSum - prevSum) == 0 or count != 10 ):
        prevSum = currSum
        count += 1
        for i in range(len(sentences)):
            inlinkContri = 0
            for j in range(len(sentences)):
                if j == i:
                    continue
                outlinkContrib = 0
                for k in range(len(sentences)):
                    if k == j:
                        continue
                    outlinkContrib += BM25_Score[j][k]
                if outlinkContrib != 0:
                    inlinkContri += (BM25_Score[j][i]/outlinkContrib) * ws[j]
                else: 
                    inlinkContri = 0
            ws[i] =  (1-d) + d * inlinkContri
            currSum += ws[i]
    return ws

In [34]:
from os import getcwd
from os.path import join
def generateIdfDict():
    idfFile = 'resources/idfFile/finalIdfscore.csv'
    file = join(getcwd(), idfFile)
    idfDict = {}
    f = open(file,'r',encoding="utf8")
    for line in f:
        lineArr = line.split(',')
        idfDict[lineArr[0]] = float(lineArr[1])
    f.close()
    print("idf done")
    return idfDict

In [35]:
import operator
def summarizeWithoutFilter(path, outPath, idf, ratio):
    b = 0.75
    k = 1.2
    d = 0.85
    sentences = readFile(path)
    stoppedSentences = stopping(sentences)
    stemmedSentences = stemming(sentences)
    summary = []
    #print(stemmedSentences)
#     idf = generateIdfDict()
    BM25_Score = createSimilarityMatrix(stemmedSentences,idf,k,b)
    sentenceScore = textRank(stemmedSentences,BM25_Score,d)
    file = open(outPath,"w+")
    n = int(ratio * len(sentences))
    
    #print(sentenceScore)
#     sentenceScore = dict(sorted(sentenceScore.items(), key=lambda x:x[1], reverse=True)[:n])
#     sentenceScore = sorted(sentenceScore.items(), key=lambda x:x[0])
    sentenceScore = sorted(sentenceScore.items(), key=lambda x:x[1], reverse=True)
#     print(sentenceScore)
    summaryIndexArray = calculateTopKSentencesWithoutFilter(sentences, sentenceScore, n)
    sortedSummaryIndexArray = sorted(summaryIndexArray, key=lambda x:x[0], reverse=False)
    for line in sortedSummaryIndexArray:
        summary.append(sentences[line[0]])
    for line in summary:
#         i = index[0]
        file.write(line + '\n')
#         file.write(sentences[i])
#         print(sentences[i])
#         file.write("\n")
#         print("\n")
    file.close()

In [36]:
import operator
def summarize(path, outPath, idf, ratio):
    b = 0.75
    k = 1.2
    d = 0.85
    sentences = readFile(path)
    stoppedSentences = stopping(sentences)
    stemmedSentences = stemming(sentences)
    summary = []
    #print(stemmedSentences)
#     idf = generateIdfDict()
    BM25_Score = createSimilarityMatrix(stemmedSentences,idf,k,b)
    sentenceScore = textRank(stemmedSentences,BM25_Score,d)
    file = open(outPath,"w+")
    n = int(ratio * len(sentences))
    
    #print(sentenceScore)
#     sentenceScore = dict(sorted(sentenceScore.items(), key=lambda x:x[1], reverse=True)[:n])
#     sentenceScore = sorted(sentenceScore.items(), key=lambda x:x[0])
    sentenceScore = sorted(sentenceScore.items(), key=lambda x:x[1], reverse=True)
#     print(sentenceScore)
    summaryIndexArray = calculateTopKSentences(sentences, sentenceScore, n)
    sortedSummaryIndexArray = sorted(summaryIndexArray, key=lambda x:x[0], reverse=False)
    for line in sortedSummaryIndexArray:
        summary.append(sentences[line[0]])
    for line in summary:
#         i = index[0]
        file.write(line + '\n')
#         file.write(sentences[i])
#         print(sentences[i])
#         file.write("\n")
#         print("\n")
    file.close()

In [37]:
# summarize("C:\\NewsData\\NewsFile4.txt","C:\\NewsData\\SummryNewsFile4.txt",0.2)


# LEX RANK

In [38]:
#  read a single file and returns fileContent and summary content  
from os import listdir,getcwd
from os.path import isfile, join
def readFromFile(filename):
#     docSummaryDict = {}
#     docSentencesDict = {}
    fileContent = []
    summaryContent = []
    summaryline = False
    for line in open(filename,'r'): 
        if line != '\n':
            if '@highlight' in line:
                summaryline = True
                continue
            if not summaryline:
                fileContent.append(line.strip())
            else:
                summaryline = False
                summaryContent.append(line)
    return fileContent, summaryContent 

In [39]:
# function to read stop words from a text file
def readStopWords(filename):
    text = []
    if(filename.endswith('txt')):
        file = open(filename, 'r')
        for line in file:
            text.append(line.strip())
        return text
    else:
        return None;


from nltk import word_tokenize
def removeStopWordsFromFile(fileContentArray):
 #  list of stopWords ( removed following words from list -> eight, eleven, fifteen, first, five, forty, four, nine, 
#   one, six, sixty, twelve, twenty, two, ten, )
# http://xpo6.com/download-stop-word-list/
    stopWordsFile = 'stop-word-list.txt'
    stopWords = readStopWords(stopWordsFile)
    if stopWords is None:
        raise Exception('Couldn\'t parse the given file. Stop Words list is empty. Please provide a text file to parse.')
    modifiedTextArr = []
    for text in fileContentArray:
        modifiedText = ''
        words = word_tokenize(text)
        for word in words:
            if word not in stopWords:
                modifiedText += word + ' '
        modifiedTextArr.append(modifiedText)
    return modifiedTextArr 

In [40]:
# nltk.download()
# creating stemmed text and stemmedTextDict(for term frequency)
from nltk.stem import PorterStemmer
from nltk import word_tokenize
def getStemmedTextForFile(fileContentArray):
    ps = PorterStemmer()
    modifiedTextArr = []
    for text in fileContentArray:
        stemmedText = ''
        words = word_tokenize(text)
        for word in words:
            stemmedWord = ps.stem(word)
            stemmedText += stemmedWord + ' '
        modifiedTextArr.append(stemmedText)
    return modifiedTextArr

In [41]:
from nltk import word_tokenize
from collections import Counter
from math import pow, log, sqrt
from nltk import word_tokenize
def idfModifiedCosine(sentence1,sentence2, idfDict):
#     print('sentence1 is ' + sentence1)
#     print('sentence2 is ' + sentence2)
    sent1TFDict = Counter()
    sent2TFDict = Counter()
#     idfDict = {}
    wordsInbothSentences = None
    sentence1Arr = word_tokenize(sentence1)
    sentence2Arr = word_tokenize(sentence2)
    for word in sentence1Arr:
        sent1TFDict[word] += 1
        if wordsInbothSentences is None:
            wordsInbothSentences = {word}
        else:
            wordsInbothSentences = wordsInbothSentences | {word}
#         if word not in idfDict:
# #             print('calculate idf for word : ' +  word + ' ' +  str(termDocfreqDict[word]))
#             idfDict[word] = log(totalNumberOfDocs / termDocfreqDict[word])
    for word in sentence2Arr:
        sent2TFDict[word] += 1
        if wordsInbothSentences is None:
            wordsInbothSentences = {word}
        else:
            wordsInbothSentences = wordsInbothSentences | {word}
#         if word not in idfDict:
# #             print('calculate idf for word : ' +  word + ' ' +  str(termDocfreqDict[word]))
#             idfDict[word] = log(totalNumberOfDocs / termDocfreqDict[word])
    num  = 0
    denSent1 = 0
    denSent2 = 0
    for word in wordsInbothSentences:
        num += sent1TFDict.get(word,0) * sent2TFDict.get(word,0) * pow(idfDict.get(word,1),2)
        denSent1 += pow(sent1TFDict.get(word,0) * idfDict.get(word,1),2)
        denSent2 += pow(sent2TFDict.get(word,0) * idfDict.get(word,1),2)              
    return num / (sqrt(denSent1) * sqrt(denSent2))

In [42]:
#  implementing matrix Product
import numpy as np
def matrixProduct(matA, matB):
    return np.matmul(matA, matB)

In [43]:
import numpy as np
def matrixTranspose(mat):
    return np.transpose(mat)

In [44]:
import numpy as np
def matrixDifference(mat1,mat2):
    return np.subtract(mat1,mat2)

In [45]:
from math import pow
# http://www.personal.soton.ac.uk/jav/soton/HELM/workbooks/workbook_30/30_4_matrx_norms.pdf
def calculateEucledeanNorm(mat):
    val = 0
    for row in mat:
        for col in range(len(row)):
            val += pow(row[col],2)
    return pow(val,1/2)

In [46]:
#  implementing power method
def powerMethod(cosineMatrix, N, tolerance):
    cosineMatrixTranspose = matrixTranspose(cosineMatrix)
    initializeP = [[(1/N) for x in range(N)] for y in range(N)]
    t = 0
    currentP = None
    while True:
        if currentP is None:
            oldP = initializeP
        else:
            oldP = currentP
        t += 1
        currentP = matrixProduct(cosineMatrixTranspose,oldP)
        differenceMatrix = matrixDifference(currentP, oldP)
        difference = calculateEucledeanNorm(differenceMatrix)      
        if difference < tolerance:
            return currentP  

In [47]:
#  Lex Rank
#  need to check from where do we get tolerance, threshold
#  idf-modified-cosine
#  We might get divide by zero error, need to think how to avoid that.

def lexRank(sentences,threshold, tolerance, idfDict):
    n = len(sentences)
    cosineMatrix = [[0 for x in range(n)] for y in range(n)]
    degree = [1 for x in range(n)]
    for i in range(0,n):
        for j in range(0,n):
            cosineMatrix[i][j] = idfModifiedCosine(sentences[i],sentences[j], idfDict)
            if cosineMatrix[i][j] > threshold:
                cosineMatrix[i][j] = 1
                degree[i] += 1
            else:
                cosineMatrix[i][j] = 0
    for i in range(0,n):
        for j in range(0,n):
            cosineMatrix[i][j] = cosineMatrix[i][j]/degree[i]
    L = powerMethod(cosineMatrix, n, tolerance)
    return L, degree

In [48]:
#  storing summarisation to disk
from os import listdir,getcwd
from os.path import isfile, join
def writeResultsToFile(filename, summarySentences):
    if(len(summarySentences)>0):
        f = open(filename,'w')
        for line in summarySentences:
            f.write(line + '\n')
        f.close()

In [49]:
from nltk import word_tokenize
def computeLogEntropy(probDict, sentence):
    words = word_tokenize(sentence)
    entropySum = 0
    for word in words:
        entropySum += -1 * (probDict[word] * log(probDict[word]))
    a = round(entropySum,1)
#     print(a)
    return a

In [50]:
from collections import Counter
from nltk import word_tokenize
def getSentencesEntropy(fileContentArray, scoreIndexArraySorted):
    entropyIndexedSentenceArray = []
    probDict = {}
    count = 0
    for sentence in fileContentArray:
        for word in word_tokenize(sentence):
            if word in probDict:
                probDict[word] += 1
            else:
                probDict[word] = 1
            count += 1
    for word in probDict:
        probDict[word] = probDict[word]/count

    for index, score in scoreIndexArraySorted:
        sentence = fileContentArray[index]
        entropyIndexedSentenceArray.append((index, score, computeLogEntropy(probDict, sentence)))
    return entropyIndexedSentenceArray

In [51]:
def calculateTopKSentencesWithoutFilter(fileContentArray, scoreIndexArraySorted, summaryCount):
#     scoreIndexArraySorted = getSentencesEntropy(fileContentArray, scoreIndexArraySorted)
    index = 0
    summaryIndexArray=[]
#     entropyScore = []
    for line in scoreIndexArraySorted:
        if index < summaryCount:
#             if format(line[2],'.2f') in entropyScore:
#                 continue
            summaryIndexArray.append(line)
            index += 1
#             entropyScore.append(format(line[2],'.2f'))
        else:
            break;
    return summaryIndexArray

In [52]:
def calculateTopKSentences(fileContentArray, scoreIndexArraySorted, summaryCount):
    scoreIndexArraySorted = getSentencesEntropy(fileContentArray, scoreIndexArraySorted)
#     s = sorted(scoreIndexArraySorted, key=lambda x:x[2], reverse=True)
#     print(s)
    index = 0
    summaryIndexArray=[]
    entropyScore = []
    for line in scoreIndexArraySorted:
        if index < summaryCount:
            if line[2] in entropyScore:
                continue
            summaryIndexArray.append(line)
            index += 1
            entropyScore.append(line[2])
        else:
            break;
    return summaryIndexArray

In [53]:
def lexSummariesWithoutFilter(inputFile, outputFile, outputHumanSummary, idfDict, cutoff = .20,threshold=0.1, tolerance=0.8):
    fileContentArray, summaryContentArray = readFromFile(inputFile)
    fileContentArrayUnStopped = removeStopWordsFromFile(fileContentArray)
    fileContentArrayStemmed = getStemmedTextForFile(fileContentArrayUnStopped)
    L, degree = lexRank(fileContentArrayStemmed,threshold, tolerance, idfDict)
    scoreIndexArray = []
    summarySentences = []
    index=0
    for row in L:
        scoreIndexArray.append((index, row[0]))
        index += 1
        
    scoreIndexArraySorted = sorted(scoreIndexArray, key=lambda x:x[1], reverse=True)
    summaryCount = int (len(fileContentArray) * cutoff)

    summaryIndexArray = calculateTopKSentencesWithoutFilter(fileContentArray, scoreIndexArraySorted, summaryCount)
    sortedSummaryIndexArray = sorted(summaryIndexArray, key=lambda x:x[0], reverse=False)
    for line in sortedSummaryIndexArray:
        summarySentences.append(fileContentArray[line[0]])
        
    writeResultsToFile(outputFile, summarySentences)
    writeResultsToFile(outputHumanSummary, summaryContentArray)

In [54]:
def lexSummaries(inputFile, outputFile, outputHumanSummary, idfDict, cutoff = .20,threshold=0.1, tolerance=0.8):
    fileContentArray, summaryContentArray = readFromFile(inputFile)
    fileContentArrayUnStopped = removeStopWordsFromFile(fileContentArray)
    fileContentArrayStemmed = getStemmedTextForFile(fileContentArrayUnStopped)
    L, degree = lexRank(fileContentArrayStemmed,threshold, tolerance, idfDict)
    scoreIndexArray = []
    summarySentences = []
    index=0
    for row in L:
        scoreIndexArray.append((index, row[0]))
        index += 1
        
    scoreIndexArraySorted = sorted(scoreIndexArray, key=lambda x:x[1], reverse=True)
    summaryCount = int (len(fileContentArray) * cutoff)

    summaryIndexArray = calculateTopKSentences(fileContentArray, scoreIndexArraySorted, summaryCount)
    sortedSummaryIndexArray = sorted(summaryIndexArray, key=lambda x:x[0], reverse=False)
    for line in sortedSummaryIndexArray:
        summarySentences.append(fileContentArray[line[0]])
    
    writeResultsToFile(outputFile, summarySentences)
    writeResultsToFile(outputHumanSummary, summaryContentArray)

In [55]:
# lexSummaries(inputFile, outputFile, outputHumanSummary, cutoff = 20,threshold=0.1, tolerance=0.1)

# Rouge Score Calculation

In [56]:
from os import listdir,getcwd
from os.path import isfile, join
def readResults(filename):
    allSentences = []
    f = open(filename,'r')
    for line in f:
        if line != '\n':
            allSentences.append(line.strip())
    f.close()
    return allSentences

In [57]:
from nltk import word_tokenize
def getUnigrams(textArr):
    text = ''.join(textArr)
    unigrams = word_tokenize(text)
    return unigrams

In [58]:
#  implementing ROGUE score
from nltk import bigrams
def getBigrams(textArr):
    bigrm = list(bigrams(getUnigrams(textArr)))
    return bigrm

In [59]:
def LCS(sentence1,sentence2):
    m = len(sentence1)
    n = len(sentence2)
    b = [[0 for i in range(0,n+1)] for j in range(0,m+1)]
    c = [[0 for i in range(0,n+1)] for j in range(0,m+1)]
    for i in range(1,m+1):
        for j in range(1,n+1):
            if sentence1[i-1] == sentence2[j-1]:
                c[i][j] = c[i-1][j-1]+1
                b[i][j] = 1
            elif c[i-1][j] >= c[i][j-1]:
                c[i][j] = c[i-1][j]
                b[i][j] = 2
            else:
                c[i][j] = c[i][j-1]
                b[i][j] = 3
    return c[m][n]

In [60]:
def LCSScore(refSummary, candidateSummary):
    lscscore = LCS(refSummary,candidateSummary)
    RLCS = lscscore/len(refSummary)
    return RLCS

In [61]:
def computeRougeLScore(candidateSummary, refSummary):
    score = 0
    count = 0
    for systemLine in candidateSummary:
        for humanLine in refSummary:
            count += 1
            score += LCSScore(humanLine, systemLine)   
    RougeL = score/count
    return RougeL

In [62]:
def computeRouge2Score(candidateSummaryBigrams, refSummaryBigrams):
    overlappingLexBigrams = []
    for bigram in candidateSummaryBigrams:
        if bigram in refSummaryBigrams:
            overlappingLexBigrams.append(bigram)
    Rouge2Lex = len(overlappingLexBigrams)/len(refSummaryBigrams)
    return Rouge2Lex

In [63]:
def computeRougeScore(lexRankOutputFile, textRankOutputFile, humanGeneratedFile):
    systemGeneratedLex = readResults(lexRankOutputFile)
    systemGeneratedText = readResults(textRankOutputFile)
    humanGenerated = readResults(humanGeneratedFile)

    # for ROUGE-2
    systemGeneratedLexBigrams = getBigrams(systemGeneratedLex)
    systemGeneratedTextBigrams = getBigrams(systemGeneratedText)
    humanGeneratedBigrams = getBigrams(humanGenerated)

    Rouge2Lex = computeRouge2Score(systemGeneratedLexBigrams, humanGeneratedBigrams)
    Rouge2Text = computeRouge2Score(systemGeneratedTextBigrams, humanGeneratedBigrams)
    
#     for ROUGE-L Score
    RougeLLex = computeRougeLScore(systemGeneratedLex, humanGenerated)
    RougeLText = computeRougeLScore(systemGeneratedText, humanGenerated)

#     summaryQualityLex = (RougeLLex+Rouge2Lex)/2
#     summaryQualityText = (RougeLText+Rouge2Text)/2

    return Rouge2Lex, Rouge2Text, RougeLLex, RougeLText

In [64]:
def computeLexSummaryQuality(lexRankOutputFile, humanGeneratedFile):
    systemGeneratedLex = readResults(lexRankOutputFile)
    humanGenerated = readResults(humanGeneratedFile)

    # for ROUGE-2
    systemGeneratedLexBigrams = getBigrams(systemGeneratedLex)
    humanGeneratedBigrams = getBigrams(humanGenerated)

    Rouge2Lex = computeRouge2Score(systemGeneratedLexBigrams, humanGeneratedBigrams)
    
#     for ROUGE-L Score
    RougeLLex = computeRougeLScore(systemGeneratedLex, humanGenerated)

    summaryQualityLex = (RougeLLex+Rouge2Lex)/2

    return summaryQualityLex

# Run Summarizer with 2 filter

In [65]:
idfDict = generateIdfDict()

idf done


In [75]:
from os import listdir,getcwd
from os.path import isfile, join

inputPath = 'resources/InputFiles/'
outputLexPath = 'resources/OutputFiles/LexRankSummary/'
outputTextPath = 'resources/OutputFiles/TextRankSummary/'
outputHumanPath = 'resources/HumanSummaries/'
# outputQualityFileName = 'resources/OutputFiles/qualityScores.txt'
rouge2Filename = 'resources/OutputFiles/rouge2scores.txt'
rougeLFilename = 'resources/OutputFiles/rougeLscores.txt'

inputFilePath = join(getcwd(), inputPath)
outputRouge2FilePath = join(getcwd(), rouge2Filename)
outputRougeLFilePath = join(getcwd(), rougeLFilename)
cutOff = .20

allFiles = [f for f in listdir(inputFilePath) if not (f.startswith('.')) and isfile(join(inputFilePath, f))]
# qualityScore = []
rouge2score = []
rougeLscore = []
index = 0
for file in allFiles:
    try: 
        inputFileName = join(inputFilePath, file)
        outputLexFileName = join(getcwd(), outputLexPath, file)
        outputTextFileName = join(getcwd(), outputTextPath, file)
        outputHumanFileName = join(getcwd(), outputHumanPath, file)
    #     Lex Rank
        lexSummaries(inputFileName, outputLexFileName, outputHumanFileName, idfDict, cutOff)
    #     Text Rank
        summarize(inputFileName,outputTextFileName,idfDict, cutOff)
    #     quality score
        Rouge2Lex, Rouge2Text, RougeLLex, RougeLText = computeRougeScore(outputLexFileName, outputTextFileName, outputHumanFileName)
        rouge2score.append((file, Rouge2Lex,Rouge2Text))
        rougeLscore.append((file, RougeLLex,RougeLText))
        index += 1
    except:
        print('file number ' + str(index) + ' name '+ str(file) + ' got an exception')
        continue
        
rouge2File = open(outputRouge2FilePath,'w')
for scores in rouge2score:
    rouge2File.write(scores[0] + ' , ' + str(scores[1]) + ' , ' + str(scores[2]) + '\n')
rouge2File.close()

rougeLFile = open(outputRougeLFilePath, 'w')
for scores in rougeLscore:
    rougeLFile.write(scores[0] + ' , ' + str(scores[1]) + ' , ' + str(scores[2]) + '\n')
rougeLFile.close()

# Code to convert Text Summary to Audio Summary

In [67]:
from gtts import gTTS
import os
def textToSpeech(text, outputFile):
    tts = gTTS(text=text, lang='en')
    tts.save(outputFile)

In [68]:
def readSummary(inputFileName):
    text = ''
    f = open(inputFileName,'r')
    for line in f:
        if line != '\n':
            text += line.strip()
    return text

In [69]:
from os import listdir,getcwd
from os.path import isfile, join
inputLexPath = 'resources/OutputFiles/LexRankSummary/'
inputTextPath = 'resources/OutputFiles/TextRankSummary/'
outputLexPath = 'resources/OutputFiles/LexRankAudio/'
outputTextPath = 'resources/OutputFiles/TextRankAudio/'
inputLexFilePath = join(getcwd(), inputLexPath)
inputTextFilePath = join(getcwd(), inputTextPath)
allLexFiles = [f for f in listdir(inputLexFilePath) if not (f.startswith('.')) and isfile(join(inputLexFilePath, f))]
allTextFiles = [f for f in listdir(inputTextFilePath) if not (f.startswith('.')) and isfile(join(inputTextFilePath, f))]
for file in allLexFiles:
    try: 
        inputFileName = join(inputLexFilePath, file)
        text = readSummary(inputFileName)
        textToSpeech(text,outputLexPath+file.split('.txt')[0]+'.mp3')
    except:
        print('file ' + file + ' got an exception')
        continue
for file in allTextFiles:
    try: 
        inputFileName = join(inputTextFilePath, file)
        text = readSummary(inputFileName)
        textToSpeech(text,outputTextPath+file.split('.txt')[0]+'.mp3')
    except:
        print('file ' + file + ' got an exception')
        continue

#  Code to convert Text Summary to Braille Text

In [70]:
def generateBrailleDict():
    # ASCII
    asciicodes = [' ','!','"','#','$','%','&','(',')','*','+',',','-','.','/','0','1','2','3','4','5','6',
                  '7','8','9',':',';','<','=','>','?','@','a','b','c','d','e','f','g','h','i','j','k','l',
                  'm','n','o','p','q','r','s','t','u','v','w','x','y','z','[','\\',']','^','_']
    # Braille symbols
    brailles = [' ','⠮','⠐','⠼','⠫','⠩','⠯','⠷','⠾','⠡','⠬','⠠','⠤','⠨','⠌','⠴','⠂','⠆','⠒','⠲','⠢','⠖',
                '⠶','⠦','⠔','⠱','⠰','⠣','⠿','⠜','⠹','⠈','⠁','⠃','⠉','⠙','⠑','⠋','⠛','⠓','⠊','⠚','⠅','⠇',
                '⠍','⠝','⠕','⠏','⠟','⠗','⠎','⠞','⠥','⠧','⠺','⠭','⠽','⠵','⠪','⠳','⠻','⠘','⠸']
    translateDict = {}
    for index,chr  in enumerate(asciicodes):
    #     print(chr,len(chr))
        translateDict[chr] =  brailles[index]
        transtab = str.maketrans(translateDict)
    return transtab

In [71]:
def textToBraille(fileName, outputPath):
    transtab=generateBrailleDict()
    file = open(fileName,'r')
    file2 = open(outputPath + fileName.split('/')[-1],'w', encoding='utf-16')
    for line in file:
        line = line.lower()
        file2.write(line.translate(transtab))
    file2.close()

In [72]:
from os import listdir,getcwd
from os.path import isfile, join
inputLexPath = 'resources/OutputFiles/LexRankSummary/'
inputTextPath = 'resources/OutputFiles/TextRankSummary/'
outputLexPath = 'resources/OutputFiles/LexRankBraille/'
outputTextPath = 'resources/OutputFiles/TextRankBraille/'
inputLexFilePath = join(getcwd(), inputLexPath)
inputTextFilePath = join(getcwd(), inputTextPath)
allLexFiles = [f for f in listdir(inputLexFilePath) if not (f.startswith('.')) and isfile(join(inputLexFilePath, f))]
allTextFiles = [f for f in listdir(inputTextFilePath) if not (f.startswith('.')) and isfile(join(inputTextFilePath, f))]
for file in allLexFiles:
    try: 
        inputFileName = join(inputLexFilePath, file)
        textToBraille(inputFileName,outputLexPath)
    except:
        print('file ' + file + ' got an exception')
        continue
        
for file in allTextFiles:
    try: 
        inputFileName = join(inputTextFilePath, file)
        textToBraille(inputFileName,outputTextPath)
    except:
        print('file ' + file + ' got an exception')
        continue